2 Python Programming 

2.1 Checksum 
Write a method that computes and returns the checksum of a message. The input message
contains four segments and each segment represents a 8-bit binary data.

1. In the method, make sure that each step is printed out in the console. For example:
    
1: check_sum = calculate_checksum ( message )
2: console >> var 1 00000000
3: var 2 10011001
4: sum 10011001
5: ...
6: checksum 10001001

2. Fill out the following table with the checksum values that are computed by the
method.

Messages Checksum
[0b01101001,0b00100000,0b01101100,0b01101111]
[0b01110110,0b01100101,0b00100000,0b01110111]
[0b01100101,0b01100010,0b00100000,0b01110011]
Table 1: Inputs for the checksum method

3. Write another method that validates or rejects given data and checksum. For the
corrupted data, the method should output an error message

1: validate_data ( message , check_sum )
2: console >> Corrupted data

In [ ]:
initial_bit_size = 10
formatted_bit_size = initial_bit_size - 2
perfect_sum = "11111111"  # same size bit as 'formatted_bit_size'
overflow_bit = "0b00000001"  # same size bit as 'initial_bit_size'
initial_sum = "0b00000000"  # same size bit as 'initial_bit_size'


# Method to calculate checksum
def calculate_checksum(message):
    sum = initial_sum
    for bit in message:
        bin_str = bin(bit)
        print_variables(sum, 1)
        print_variables(bin_str, 2)
        sum = bin(int(sum, 2) + int(bin_str, 2))

        sum = get_sum_wrapped(sum)
        print_sum(sum)
    cs = print_check_sum(get_formatted_bits(sum))
    return cs


# Method to Validate Message against checksum
def validate_data(message, check_sum):
    message.append(check_sum)
    sum = initial_sum
    for bit in message:
        bin_str = bin(bit)
        print_variables(sum, 1)
        print_variables(bin_str, 2)
        sum = bin(int(sum, 2) + int(bin_str, 2))
        sum = get_sum_wrapped(sum)
        print_sum(sum)
    validated_bits = get_formatted_bits(sum)
    if validated_bits == perfect_sum:
        print(f"Got the following sum {validated_bits} after the validation")
        return True
    else:
        print("Corrupted data")
        return False


def print_check_sum(sum):
    check_sum = ""
    for i in sum:  # Flipping values to get 1's compliment
        if i == "0":
            check_sum += "1"
        else:
            check_sum += "0"
    print(f"checksum {check_sum}")
    return check_sum


# Get the wrapped value
# If there is an overflow then
# Add it to the sum and return
def get_sum_wrapped(sum):
    x = len(sum)
    if x > initial_bit_size:
        y = sum[2:]  # removing '0b'
        y = y[1:]  # slicing the number except the overflow bit
        z = bin(int(y, 2) + int(overflow_bit, 2))  # Add overflow bit to the sum
        return z
    else:
        return sum


def print_sum(sum):
    print(f"sum   {get_formatted_bits(sum)}")


def print_variables(value, count):
    print(f"var {count} {get_formatted_bits(value)}")


def get_formatted_bits(value):
    value = value[2:]  # Slicing the variable
    x = len(value)
    if x == formatted_bit_size:
        return value
    else:
        y = formatted_bit_size - x
        zero = ""
        for k in range(0, y):
            zero += "0"
        value = zero + value
        return value


if __name__ == "__main__":
    # a = [0b1010101100110101, 0b1100001111010001, 0b0101001000110001, 0b0011000011101101] # Change Upper values

    x = [0b01101001, 0b00100000, 0b01101100, 0b01101111]
    y = [0b01110110, 0b01100101, 0b00100000, 0b01110111]
    z = [0b01100101, 0b01100010, 0b00100000, 0b01110011]

    cs_x = int(calculate_checksum(z), 2)
    validate_data(z, cs_x)


2.2 TCP Client Server 

In this task you will simulate the checksum operation. You will write a sender (sender.py)
and a receiver (receiver.py). The communication between them will be with IPv4 and
TCP.

1. sender.py generates random messages as shown Table 1. When it gets four segments
of messages, it calculates the checksum of a message with the method you will
implement for Assignment 2.1 and send them to the receiver.

2. The receiver.py performs checksum validation with the method you will implement
for Assignment 2.1 and sends the result of the validation to the sender.

3. When you execute python receiver.py from the console, you should print out the
flow of receiver similarly:

    1: Connection from (' $SERVER$ : $PORT$ ).

    2: Got the following message [219 , 118 , 233 , 178] and 17 as

    3: checksum from the sender

    4: var 1 00000000

    5: var 2 11011011

    6: sum 11011011

    7: var 1 11011011

    8: var 2 01110110

    9: sum 01010010

    10: var 1 01010010

    11: var 2 11101001

    12: sum 00111100

    13: var 1 00111100

    14: var 2 10110010

    15: sum 11101110

    16: Got the following sum 11111111 after the validation

    17: ...

4. When you execute python sender.py from the console, you should print out the
flow of sender similarly:

    1: connected to tmps - $SERVER$ : $PORT$
    
    2: var 1 00000000
    
    3: var 2 11011011
    
    4: sum 11011011
    
    5: var 1 11011011
    
    6: var 2 01110110
    
    7: sum 01010010
    
    8: var 1 01010010
    
    9: var 2 11101001
    
    10: sum 00111100
    
    11: var 1 00111100
    
    12: var 2 10110010
    
    13: sum 11101110
    
    14: checksum 00010001
    
    15: Response from the receiver b ' Receiver Message :
    
    16: Data is correctly received and checksum is 00010001 '
    
    17: ...

For the programming tasks, you can use the following libraries: pickle, random, socket.
Make sure that the messages between sender and receiver are properly printed.

Along with your code, send the print screens of the terminals.



In [ ]:
#sender
import pickle
import random
import socket

from task_2 import *

ENCODING = "utf-8"
LOCAL_HOST = "127.0.0.1"
CON_PORT = 10000

X = [0b01101001, 0b00100000, 0b01101100, 0b01101111]
Y = [0b01110110, 0b01100101, 0b00100000, 0b01110111]
Z = [0b01100101, 0b01100010, 0b00100000, 0b01110011]


def start_client():
    # Create a TCP/IP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Connect the socket to the port where the server is listening
    server_address = (LOCAL_HOST, CON_PORT)
    print(f'connected to tmps-{LOCAL_HOST}:{CON_PORT}')
    sock.connect(server_address)

    # Generating four 8-bit values
    message_list = random.sample(range(0, 256), 4)

    check_sum_int = int(calculate_checksum(message_list), 2)
    message_list.append(check_sum_int)
    try:
        # Send Data to Server
        send_data(sock, message_list)
    finally:
        while True:
            ack = sock.recv(300)
            #ack = ack.decode(ENCODING)
            if ack == "exit".encode(ENCODING):
                break
            else:
                print(f"Response from the receiver {ack}")
    # Close Connection
    sock.close()


def send_data(sock, message):
    # Send data
    data = pickle.dumps(message)  # Using pickle to send 'list' as payload
    sock.sendall(data)


if __name__ == "__main__":
    start_client()


In [ ]:
#WS_2_receiver.py

import pickle
import socket

from task_2 import *

LOCAL_HOST = "127.0.0.1"
CON_PORT = 10000


def start_server():
    # Create a TCP/IP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Bind the socket to the port
    server_address = (LOCAL_HOST, CON_PORT)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # Using this to reuse same 'server_address'
    sock.bind(server_address)
    # Listen for incoming connections
    sock.listen(1)

    while True:
        # Wait for a connection
        # print('waiting for a connection')
        connection, client_address = sock.accept()
        try:
            in_con = client_address[0]
            in_port = client_address[1]
            in_server = f"{in_con}:{in_port}"
            print(f'connection from {in_server}')

            # Receive the data in small chunks and retransmit it
            while True:
                data = connection.recv(48)
                data = pickle.loads(data)  # Using pickle to get 'list' out of received payload
                messages = data[0:4]
                check_sum = data[4]
                print(f"Got the following message {messages} and {check_sum} as checksum from the sender")
                is_valid = validate_data(messages, check_sum)  # Validate message against checksum
                if data:
                    if is_valid:
                        check_sum = bin(check_sum)
                        final_message = "Receiver Message : Data is correctly received and checksum is "
                        final_message += get_formatted_bits(check_sum)
                        connection.sendall(bytes(final_message, 'utf-8'))  # Send success message
                    else:
                        final_message = "Corrupted data"
                        connection.sendall(bytes(final_message, 'utf-8'))
                    connection.sendall(bytes("exit", 'utf-8'))  # Sending signal to close
                else:
                    print(f'no more data from {client_address}')
                    break
        finally:
            # Clean up the connection
            connection.close()
            break


if __name__ == "__main__":
    start_server()
